In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#Artificial dataset, lets say XOR
def xor():
    X = np.array([[0,1], [1,0], [1,1],[0,0]], dtype=float)
    Y = np.array([1,1,0,0], dtype=float).reshape((-1,1))
    return X,Y

def square_function():
    X = []
    Y = []
    for x in range(10):
        X.append(x)
        Y.append(x**2)
    return np.array(X,dtype=np.float32).reshape(-1,1),np.array(Y, dtype=np.float32).reshape(-1,1)

def high_dim():
    X = []
    Y = []
    for x in range(100):
        X.append([x, x+3, x+4, x+5, x+10, x+20, x+60])
    return np.array(X,dtype=np.float32).reshape(-1,7), None



# Simple autoencoder

So basically the only thing that is changing is the loss function, we want to get the same output as the input. This can be used to reduce the dimensionality of the input (encoding) with hidden layers, then we get the input back by decoding later, this is what the network is training.

In [7]:
#input

X,Y = high_dim()

batch_size = 10

#Helper functions
def fully_connected(x, size, activation=tf.nn.relu):
    b = tf.Variable(tf.zeros([size]))
    weights = tf.Variable(tf.truncated_normal([x.get_shape()[1].value, size]))
    return activation(b + tf.matmul(x, weights))


INPUT_SHAPE = X.shape

x = tf.placeholder("float", [None,X.shape[1]])

#simple neural network to solve the xor problem

h1 = fully_connected(x, 20, activation=tf.nn.tanh)
print(h1.get_shape())

h2 = fully_connected(h1, 20)
print(h2.get_shape())

h3 = fully_connected(h2, 4)
print(h3.get_shape())

h4 = fully_connected(h3, 20)
print(h4.get_shape())

out = fully_connected(tf.nn.dropout(h4, 0.5), X.shape[1])
print(out.get_shape())

loss = tf.reduce_mean((out-x)**2)
tf.scalar_summary("loss", loss)


optimizer = tf.train.AdamOptimizer(0.1)
train_step = optimizer.minimize(loss)



(?, 20)
(?, 20)
(?, 4)
(?, 20)
(?, 7)


# Run the session

In [ ]:
init =  tf.initialize_all_variables()

num_epochs=10000
#merge = tf.merge_all_summaries()

with tf.Session() as sess:
    
    writer = tf.train.SummaryWriter("./summary", sess.graph)
    sess.run(init)
    #sess.run(merge)

    for epoch in range(num_epochs):
        for batch in range(X.shape[0]//batch_size):
            feed_dict = {
                x : X[batch*batch_size:batch*batch_size + batch_size],
            }
            _, lossVal = sess.run([train_step, loss], feed_dict=feed_dict)
        #writer.add_summary(merged, epoch)
        if(epoch%40 == 0):
            print("%d. Epoch Loss: %.4f "  %(epoch, lossVal))
        
        

0. Epoch Loss: 7357.2871 
40. Epoch Loss: 3047.4851 
80. Epoch Loss: 2939.5378 
120. Epoch Loss: 3175.4475 
160. Epoch Loss: 2284.9050 
200. Epoch Loss: 2348.4766 
240. Epoch Loss: 2704.4995 
280. Epoch Loss: 2209.0679 
320. Epoch Loss: 2424.2141 
360. Epoch Loss: 1861.2480 
400. Epoch Loss: 2161.4360 
440. Epoch Loss: 2320.6472 
480. Epoch Loss: 2161.1194 
520. Epoch Loss: 2184.9412 
560. Epoch Loss: 2203.8164 
600. Epoch Loss: 2038.0466 
640. Epoch Loss: 2097.0957 
680. Epoch Loss: 2111.7097 
720. Epoch Loss: 2093.4980 
760. Epoch Loss: 2078.1555 
800. Epoch Loss: 2049.7087 
840. Epoch Loss: 2054.0583 
880. Epoch Loss: 2039.0243 
920. Epoch Loss: 2036.1897 
960. Epoch Loss: 2034.2491 
1000. Epoch Loss: 2034.1201 
1040. Epoch Loss: 2034.1320 
1080. Epoch Loss: 2034.0948 
1120. Epoch Loss: 2034.1000 
1160. Epoch Loss: 2034.0997 
1200. Epoch Loss: 2034.0983 
1240. Epoch Loss: 2034.0994 
1280. Epoch Loss: 2034.1021 
1320. Epoch Loss: 2034.1067 
1360. Epoch Loss: 2034.1056 
1400. Epoch Lo